In [1]:
import librosa
import numpy as np
import pandas as pd
from scipy import signal
import noisereduce as nr

from IPython.display import Audio

In [2]:
class AudioPreprocessor:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def load_audio(self, file_path):
        """Carga un archivo de audio."""
        audio, sr = librosa.load(file_path, sr=self.sample_rate)
        return audio
    
    def normalize_volume(self, audio, target_dBFS=-20):
        """Normaliza el volumen del audio a un nivel objetivo en dBFS."""
        rms = librosa.feature.rms(y=audio)[0]
        mean_rms = np.mean(rms)
        current_dBFS = 20 * np.log10(mean_rms) if mean_rms > 0 else -np.inf
        adjustment = target_dBFS - current_dBFS
        normalized_audio = audio * (10 ** (adjustment / 20))
        # Prevenir clipping
        normalized_audio = np.clip(normalized_audio, -1.0, 1.0)
        return normalized_audio
    
    def remove_silence(self, audio, top_db=30):
        """Elimina silencios al inicio y final del audio."""
        return librosa.effects.trim(audio, top_db=top_db)[0]
    
    def apply_bandpass_filter(self, audio, lowcut=80, highcut=8000):
        """Aplica un filtro paso banda para reducir ruido."""
        nyquist = self.sample_rate // 2
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = signal.butter(4, [low, high], btype='band')
        return signal.filtfilt(b, a, audio)
    
    def reduce_noise(self, audio, noise_audio=None):
        """Reduce el ruido de un audio utilizando un perfil de ruido."""
        if noise_audio is None:
            noise_audio = audio[:int(len(audio) * 0.1)]
        
        reduced_audio = nr.reduce_noise(y=audio, y_noise=noise_audio, sr=self.sample_rate)
        return reduced_audio
    
    def clip_prevention(self, audio, threshold=0.95):
        """Previene el clipping manteniendo la señal dentro de los límites."""
        max_val = np.max(np.abs(audio))
        if max_val > threshold:
            audio = audio * (threshold / max_val)
        return audio

In [3]:
df = pd.read_csv('dataset.csv')

# Seleccionar una fila específica
row_index = 900
audio_path = df.loc[row_index, 'ruta_archivo']

print(f"Audio seleccionado: {audio_path}")
# Inicializar el preprocesador
processor = AudioPreprocessor()

Audio seleccionado: audios/no/Diego_no_083.wav


In [4]:
# Cargar el audio
audio = processor.load_audio(audio_path)

# Reproducir el audio original
print("Audio original:")
Audio(audio, rate=processor.sample_rate)

Audio original:


In [5]:
# Eliminar silencios
audio_silence_removed = processor.remove_silence(audio)
print("Audio sin silencios:")
Audio(audio_silence_removed, rate=processor.sample_rate)

Audio sin silencios:


In [6]:
# Normalizar el volumen
audio_normalized = processor.normalize_volume(audio_silence_removed)
print("Audio normalizado:")
Audio(audio_normalized, rate=processor.sample_rate)

Audio normalizado:


In [7]:
# Aplicar filtro paso banda
audio_filtered = processor.apply_bandpass_filter(audio_normalized)
print("Audio filtrado:")
Audio(audio_filtered, rate=processor.sample_rate)

Audio filtrado:


In [8]:
# Reducir ruido
audio_noise_reduced = processor.reduce_noise(audio_filtered)
print("Audio con reducción de ruido:")
Audio(audio_noise_reduced, rate=processor.sample_rate)

Audio con reducción de ruido:


In [9]:
# Prevención de clipping
audio_clipped = processor.clip_prevention(audio_noise_reduced)
print("Audio final (prevención de clipping):")
Audio(audio_clipped, rate=processor.sample_rate)

Audio final (prevención de clipping):
